In [ ]:
!git clone https://github.com/muxspace/facial_expressions

In [1]:
import csv
data = {}
with open('./facial_expressions/data/legend.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        key = row[2].lower()
        if key in data:
            data[key].append(row[1])
        else:
            data[key] = [row[1]]

In [2]:
list_of_emotions = list(data.keys())
list_of_emotions

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [3]:
for e, i in data.items():
    print(e, len(i))

anger 252
surprise 368
disgust 208
fear 21
neutral 6868
happiness 5696
sadness 268
contempt 9


In [5]:
import os

os.mkdir('sorted_data')
os.mkdir('sorted_data/training')
os.mkdir('sorted_data/testing')

In [7]:
for emotion in list_of_emotions:
    os.mkdir(os.path.join('sorted_data/training/', emotion))
    os.mkdir(os.path.join('sorted_data/testing/', emotion))

In [ ]:
from shutil import copyfile

split_size = 0.8

for face_emo, photos in data.items():
    train_size = int(split_size*len(photos))
    train_images = photos[:train_size]
    test_images = photos[train_size:]
    
    for photo in train_images:
        s = os.path.join('./facial_expressions/images', photo)
        d = os.path.join('./sorted_data/training', face_emo, photo)
        copyfile(s, d)
    for photo in test_images:
        s = os.path.join('./facial_expressions/images', photo)
        d = os.path.join('./sorted_data/testing', face_emo, photo)
        copyfile(s, d)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

2023-04-09 10:13:16.969579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Method - 1

In [10]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])

model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
print(model.summary())

train_dir = './sorted_data/training'
test_dir = './sorted_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

2023-04-09 10:13:36.073290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 10, 64)       0

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/218315034.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 40s 457ms/step - loss: 0.9009 - acc: 0.6324 - val_loss: 1.3300 - val_acc: 0.6360
Epoch 2/10
86/86 [==============================] - 40s 463ms/step - loss: 0.5554 - acc: 0.8087 - val_loss: 1.5537 - val_acc: 0.6689
Epoch 3/10
86/86 [==============================] - 39s 455ms/step - loss: 0.4986 - acc: 0.8261 - val_loss: 1.3733 - val_acc: 0.6947
Epoch 4/10
86/86 [==============================] - 40s 467ms/step - loss: 0.4469 - acc: 0.8466 - val_loss: 1.5817 - val_acc: 0.6867
Epoch 5/10
86/86 [==============================] - 42s 492ms/step - loss: 0.4282 - acc: 0.8529 - val_loss: 1.7734 - val_acc: 0.6718


# Method - 2 - Data Augmentation

In [12]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])

model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
print(model.summary())

datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    class_mode='categorical',
    batch_size=128
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(100, 100),
    class_mode='categorical',
    batch_size=128
)

es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 10, 10, 64)      

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/3543237801.py:50: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
342/342 [==============================] - 99s 287ms/step - loss: 1.0053 - acc: 0.4977 - val_loss: 1.7377 - val_acc: 0.4788
Epoch 2/10
342/342 [==============================] - 78s 229ms/step - loss: 0.9747 - acc: 0.5037 - val_loss: 1.8217 - val_acc: 0.4880
Epoch 3/10
342/342 [==============================] - 81s 236ms/step - loss: 0.9680 - acc: 0.5145 - val_loss: 2.5707 - val_acc: 0.4001


# Method - 3 - Increasing Model Capacity

In [14]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential([
      Conv2D(32, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(128, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(512, activation='relu'),
      Dropout(0.5),
      Dense(8, activation='softmax')
])

model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
print(model.summary())

es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 49, 49, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 23, 23, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 10, 10, 128)     

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/186797225.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 89s 1s/step - loss: 0.9969 - acc: 0.5612 - val_loss: 1.7051 - val_acc: 0.6324
Epoch 2/10
86/86 [==============================] - 95s 1s/step - loss: 0.6277 - acc: 0.7880 - val_loss: 1.7437 - val_acc: 0.6605
Epoch 3/10
86/86 [==============================] - 79s 919ms/step - loss: 0.5385 - acc: 0.8187 - val_loss: 1.4469 - val_acc: 0.6823
Epoch 4/10
86/86 [==============================] - 75s 875ms/step - loss: 0.4671 - acc: 0.8402 - val_loss: 1.7576 - val_acc: 0.6798
Epoch 5/10
86/86 [==============================] - 76s 879ms/step - loss: 0.4272 - acc: 0.8532 - val_loss: 1.5365 - val_acc: 0.6915


# Method - 4 : Add Regularization Techniques

In [15]:
from tensorflow.keras import regularizers

model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dropout(0.5),
      Dense(8, activation='softmax')
])

model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
print(model.summary())

es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 10, 10, 64)      

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/1579458053.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 50s 572ms/step - loss: 0.9849 - acc: 0.5797 - val_loss: 1.5588 - val_acc: 0.5759
Epoch 2/10
86/86 [==============================] - 41s 479ms/step - loss: 0.6420 - acc: 0.7863 - val_loss: 1.3889 - val_acc: 0.6492
Epoch 3/10
86/86 [==============================] - 40s 459ms/step - loss: 0.5458 - acc: 0.8202 - val_loss: 1.3409 - val_acc: 0.6721
Epoch 4/10
86/86 [==============================] - 49s 566ms/step - loss: 0.5024 - acc: 0.8344 - val_loss: 1.3549 - val_acc: 0.6864
Epoch 5/10
86/86 [==============================] - 43s 496ms/step - loss: 0.4665 - acc: 0.8427 - val_loss: 1.7084 - val_acc: 0.6659
Epoch 6/10
86/86 [==============================] - 47s 549ms/step - loss: 0.4200 - acc: 0.8564 - val_loss: 1.7731 - val_acc: 0.6758


# Method - 5 : Learning Rate Scheduling

In [16]:
from tensorflow.keras.callbacks import LearningRateScheduler

model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])

model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
print(model.summary())

es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)


def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


lr_scheduler = LearningRateScheduler(scheduler)

model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks=[lr_scheduler])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 10, 10, 64)      

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/1001216397.py:46: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 41s 470ms/step - loss: 0.9200 - acc: 0.6060 - val_loss: 1.2277 - val_acc: 0.6400 - lr: 0.0010
Epoch 2/10
86/86 [==============================] - 40s 463ms/step - loss: 0.5794 - acc: 0.8034 - val_loss: 1.4076 - val_acc: 0.6721 - lr: 0.0010
Epoch 3/10
86/86 [==============================] - 40s 463ms/step - loss: 0.4984 - acc: 0.8302 - val_loss: 1.6144 - val_acc: 0.6554 - lr: 0.0010
Epoch 4/10
86/86 [==============================] - 41s 474ms/step - loss: 0.4408 - acc: 0.8479 - val_loss: 1.4307 - val_acc: 0.6674 - lr: 0.0010
Epoch 5/10
86/86 [==============================] - 48s 553ms/step - loss: 0.4062 - acc: 0.8603 - val_loss: 2.0637 - val_acc: 0.6732 - lr: 0.0010
Epoch 6/10
86/86 [==============================] - 49s 567ms/step - loss: 0.3668 - acc: 0.8721 - val_loss: 2.1731 - val_acc: 0.6776 - lr: 0.0010
Epoch 7/10
86/86 [==============================] - 53s 621ms/step - loss: 0.3341 - acc: 0.8828 - val_loss: 1.8537 - val_acc

# Method - 6 : Different Optimiser

In [17]:
from tensorflow.keras.optimizers import RMSprop


model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])

model.compile(optimizer = RMSprop(lr=0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
print(model.summary())

es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 10, 10, 64)      

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/3704869186.py:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 48s 548ms/step - loss: 1.0815 - acc: 0.4975 - val_loss: 0.9925 - val_acc: 0.5777
Epoch 2/10
86/86 [==============================] - 45s 518ms/step - loss: 0.8499 - acc: 0.6638 - val_loss: 1.6475 - val_acc: 0.6134
Epoch 3/10
86/86 [==============================] - 41s 473ms/step - loss: 0.6653 - acc: 0.7639 - val_loss: 1.8012 - val_acc: 0.6357
Epoch 4/10
86/86 [==============================] - 45s 523ms/step - loss: 0.5782 - acc: 0.7987 - val_loss: 1.6320 - val_acc: 0.6579
Epoch 5/10
86/86 [==============================] - 46s 530ms/step - loss: 0.5423 - acc: 0.8089 - val_loss: 0.9542 - val_acc: 0.6441
Epoch 6/10
86/86 [==============================] - 42s 493ms/step - loss: 0.4763 - acc: 0.8326 - val_loss: 2.0522 - val_acc: 0.6408


# Method - 7 : Hyperparameter Tuning

In [19]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(8, activation='softmax')
])

model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
print(model.summary())

es = EarlyStopping(monitor='val_acc', patience = 3, min_delta=0.01)

check_point = ModelCheckpoint(filepath='./models/best_model.h5', save_best_only=True)

history = model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    v# VGG16alidation_data = test_generator,
                    callbacks = [es, check_point])

print(history)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 10, 10, 64)      

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/118832629.py:39: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 38s 422ms/step - loss: 0.8620 - acc: 0.6673 - val_loss: 1.5250 - val_acc: 0.6415
Epoch 2/10
86/86 [==============================] - 48s 560ms/step - loss: 0.5861 - acc: 0.7993 - val_loss: 1.8277 - val_acc: 0.6670
Epoch 3/10
86/86 [==============================] - 40s 464ms/step - loss: 0.4961 - acc: 0.8286 - val_loss: 1.8800 - val_acc: 0.6674
Epoch 4/10
86/86 [==============================] - 41s 476ms/step - loss: 0.4545 - acc: 0.8420 - val_loss: 1.6946 - val_acc: 0.6823
Epoch 5/10
86/86 [==============================] - 41s 472ms/step - loss: 0.4020 - acc: 0.8600 - val_loss: 1.5827 - val_acc: 0.6845
Epoch 6/10
86/86 [==============================] - 40s 469ms/step - loss: 0.3680 - acc: 0.8714 - val_loss: 2.3603 - val_acc: 0.6780
Epoch 7/10
86/86 [==============================] - 39s 454ms/step - loss: 0.3201 - acc: 0.8879 - val_loss: 2.3485 - val_acc: 0.6645


# Method 8.1 : Transfer learning - VGG16

In [49]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(input_shape=(100, 100, 3), include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(8, activation='softmax')
])

model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

es = EarlyStopping(monitor='val_acc', patience=3, min_delta=0.01)


def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


lr_scheduler = LearningRateScheduler(scheduler)

model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es, lr_scheduler])


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten_23 (Flatten)        (None, 4608)              0         
                                                                 
 dense_53 (Dense)            (None, 1024)              4719616   
                                                                 
 dense_54 (Dense)            (None, 8)                 8200      
                                                                 
Total params: 19,442,504
Trainable params: 4,727,816
Non-trainable params: 14,714,688
_________________________________________________________________


/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/1084309676.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


None
Epoch 1/10
86/86 [==============================] - 417s 5s/step - loss: 1.1863 - acc: 0.6060 - val_loss: 1.0130 - val_acc: 0.5667 - lr: 0.0010
Epoch 2/10
86/86 [==============================] - 421s 5s/step - loss: 0.7753 - acc: 0.6950 - val_loss: 1.0917 - val_acc: 0.5890 - lr: 0.0010
Epoch 3/10
86/86 [==============================] - 422s 5s/step - loss: 0.7384 - acc: 0.7111 - val_loss: 1.1530 - val_acc: 0.5646 - lr: 0.0010
Epoch 4/10
86/86 [==============================] - 423s 5s/step - loss: 0.7052 - acc: 0.7271 - val_loss: 1.1327 - val_acc: 0.5839 - lr: 0.0010
Epoch 5/10
86/86 [==============================] - 425s 5s/step - loss: 0.7373 - acc: 0.7117 - val_loss: 1.1111 - val_acc: 0.6061 - lr: 0.0010
Epoch 6/10
86/86 [==============================] - 424s 5s/step - loss: 0.6802 - acc: 0.7390 - val_loss: 1.0836 - val_acc: 0.5875 - lr: 0.0010
Epoch 7/10
86/86 [==============================] - 425s 5s/step - loss: 0.6832 - acc: 0.7346 - val_loss: 1.2806 - val_acc: 0.5729 

# Method 8.2 : Transfer learning - ResNet50

In [52]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(100, 100, 3), include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(8, activation='softmax')
])

model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

es = EarlyStopping(monitor='val_acc', patience=3, min_delta=0.01)


def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


lr_scheduler = LearningRateScheduler(scheduler)

model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es, lr_scheduler])


Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_26 (Flatten)        (None, 32768)             0         
                                                                 
 dense_59 (Dense)            (None, 1024)              33555456  
                                                                 
 dense_60 (Dense)            (None, 8)                 8200      
                                                                 
Total params: 57,151,368
Trainable params: 33,563,656
Non-trainable params: 23,587,712
_________________________________________________________________


/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/565419335.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


None
Epoch 1/10
86/86 [==============================] - 330s 4s/step - loss: 4.2527 - acc: 0.4419 - val_loss: 1.1656 - val_acc: 0.4559 - lr: 0.0010
Epoch 2/10
86/86 [==============================] - 289s 3s/step - loss: 1.1727 - acc: 0.4821 - val_loss: 1.4835 - val_acc: 0.4555 - lr: 0.0010
Epoch 3/10
86/86 [==============================] - 246s 3s/step - loss: 1.6348 - acc: 0.4780 - val_loss: 1.0601 - val_acc: 0.4351 - lr: 0.0010
Epoch 4/10
 3/86 [>.............................] - ETA: 3:15 - loss: 1.2806 - acc: 0.4479

KeyboardInterrupt: 

# Method 8.3 - Transfer Learning ( Inceptionv3 )

In [46]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_model = InceptionV3(include_top=False, input_shape=(100, 100, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
output = Dense(8, activation='softmax')(x)

model = Model(base_model.input, output)
model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

es = EarlyStopping(monitor='val_acc', patience=3, min_delta=0.01)


def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


lr_scheduler = LearningRateScheduler(scheduler)

model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es, lr_scheduler])


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_247 (Conv2D)            (None, 49, 49, 32)   864         ['input_11[0][0]']               
                                                                                                  
 batch_normalization_188 (Batch  (None, 49, 49, 32)  96          ['conv2d_247[0][0]']             
 Normalization)                                                                                   
                                                                                            

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_198 (Batch  (None, 10, 10, 96)  288         ['conv2d_257[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_199 (Batch  (None, 10, 10, 32)  96          ['conv2d_258[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_193 (Activation)    (None, 10, 10, 64)   0           ['batch_normalization_193[0][0]']
                                                                                                  
 activatio

 activation_210 (Activation)    (None, 10, 10, 64)   0           ['batch_normalization_210[0][0]']
                                                                                                  
 conv2d_267 (Conv2D)            (None, 10, 10, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_270 (Conv2D)            (None, 10, 10, 96)   55296       ['activation_210[0][0]']         
                                                                                                  
 batch_normalization_208 (Batch  (None, 10, 10, 48)  144         ['conv2d_267[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_211 (Batch  (None, 10, 10, 96)  288         ['conv2d_270[0][0]']             
 Normaliza

 mixed3 (Concatenate)           (None, 4, 4, 768)    0           ['activation_214[0][0]',         
                                                                  'activation_217[0][0]',         
                                                                  'max_pooling2d_65[0][0]']       
                                                                                                  
 conv2d_281 (Conv2D)            (None, 4, 4, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_222 (Batch  (None, 4, 4, 128)   384         ['conv2d_281[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_222 (Activation)    (None, 4, 4, 128)    0           ['batch_normalization_222[0][0]']
          

                                                                                                  
 batch_normalization_232 (Batch  (None, 4, 4, 160)   480         ['conv2d_291[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_232 (Activation)    (None, 4, 4, 160)    0           ['batch_normalization_232[0][0]']
                                                                                                  
 conv2d_292 (Conv2D)            (None, 4, 4, 160)    179200      ['activation_232[0][0]']         
                                                                                                  
 batch_normalization_233 (Batch  (None, 4, 4, 160)   480         ['conv2d_292[0][0]']             
 Normalization)                                                                                   
          

...

 conv2d_321 (Conv2D)            (None, 4, 4, 192)    258048      ['activation_261[0][0]']         
                                                                                                  
 batch_normalization_258 (Batch  (None, 4, 4, 192)   576         ['conv2d_317[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_262 (Batch  (None, 4, 4, 192)   576         ['conv2d_321[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_258 (Activation)    (None, 4, 4, 192)    0           ['batch_normalization_258[0][0]']
                                                                                                  
 activatio

                                                                                                  
 activation_266 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_266[0][0]']
                                                                                                  
 activation_267 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_267[0][0]']
                                                                                                  
 activation_270 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_270[0][0]']
                                                                                                  
 activation_271 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_271[0][0]']
                                                                                                  
 batch_normalization_272 (Batch  (None, 1, 1, 192)   576         ['conv2d_331[0][0]']             
 Normaliza

                                                                                                  
 activation_280 (Activation)    (None, 1, 1, 384)    0           ['batch_normalization_280[0][0]']
                                                                                                  
 batch_normalization_281 (Batch  (None, 1, 1, 192)   576         ['conv2d_340[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_273 (Activation)    (None, 1, 1, 320)    0           ['batch_normalization_273[0][0]']
                                                                                                  
 mixed9_1 (Concatenate)         (None, 1, 1, 768)    0           ['activation_275[0][0]',         
                                                                  'activation_276[0][0]']         
          

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/1963490440.py:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
86/86 [==============================] - 101s 1s/step - loss: 1.2083 - acc: 0.5688 - val_loss: 1.2841 - val_acc: 0.4292 - lr: 0.0010
Epoch 2/10
86/86 [==============================] - 86s 1s/step - loss: 0.9048 - acc: 0.6131 - val_loss: 1.2038 - val_acc: 0.5124 - lr: 0.0010
Epoch 3/10
86/86 [==============================] - 86s 997ms/step - loss: 0.8819 - acc: 0.6216 - val_loss: 1.3546 - val_acc: 0.5292 - lr: 0.0010
Epoch 4/10
86/86 [==============================] - 86s 994ms/step - loss: 0.8708 - acc: 0.6313 - val_loss: 1.2419 - val_acc: 0.5058 - lr: 0.0010
Epoch 5/10
86/86 [==============================] - 86s 996ms/step - loss: 0.8513 - acc: 0.6394 - val_loss: 1.4105 - val_acc: 0.5029 - lr: 0.0010
Epoch 6/10
86/86 [==============================] - 86s 1s/step - loss: 0.8418 - acc: 0.6468 - val_loss: 1.6597 - val_acc: 0.5310 - lr: 0.0010


# Method - 8.4 : Transfer Learning (MobileNet)

In [48]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet

# Define the input shape
input_shape = (100, 100, 3)

# Create the base MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a classification layer
predictions = Dense(8, activation='softmax')(x)

# Combine the base model and the new layers
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

# Set up the early stopping callback
es = EarlyStopping(monitor='val_acc', patience=3, min_delta=0.01)

# Set up the learning rate scheduler callback
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
model.fit_generator(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es, lr_scheduler])


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 100, 100, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 50, 50, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 50, 50, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 50, 50, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 50, 50, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 50, 50, 32)       128       
 ation)                                                    

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 6, 6, 512)        2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 6, 6, 512)         0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 6, 6, 512)        4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 6, 6, 512)        2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 6, 6, 512)         0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 6, 6, 512)         262144    
                                                                 
 conv_pw_9

/var/folders/w1/np1xc13x3fsf33wqh8hwp3fr0000gn/T/ipykernel_5769/715175190.py:44: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


None
Epoch 1/10
86/86 [==============================] - 71s 813ms/step - loss: 1.0870 - acc: 0.6774 - val_loss: 1.0961 - val_acc: 0.6050 - lr: 0.0010
Epoch 2/10
86/86 [==============================] - 69s 800ms/step - loss: 0.7135 - acc: 0.7294 - val_loss: 1.1677 - val_acc: 0.6072 - lr: 0.0010
Epoch 3/10
86/86 [==============================] - 68s 795ms/step - loss: 0.6795 - acc: 0.7434 - val_loss: 1.3965 - val_acc: 0.6036 - lr: 0.0010
Epoch 4/10
86/86 [==============================] - 68s 790ms/step - loss: 0.6544 - acc: 0.7561 - val_loss: 1.5991 - val_acc: 0.5835 - lr: 0.0010
